# Shear bias estimation

In [1]:
from shear_bias import *

## Setup

### Job control

In [2]:
job = param()

# Set re_run to True (False) if re-runs of jobs should overwrite (keep)
# previously produced output files
job.re_run = False

# Set dry_run to True (False) for jobs to run in test (real) mode
job.dry_run = False

### Shear values

In [3]:
# Small shear change for numerical derivative
dg = 0.02

In [4]:
# List of signs for shear change for the two shear components.

# Five steps (one in each direction + (0, 0)
g_steps = [(-1, 0), (0, -1), (1, 0), (0, 1), (0, 0)]

### Number of images

In [5]:
# Number of galaxy postage stamps per image is nxy_tiles^2
nxy_tiles = 2
#nxy_tiles = 100

# Number of files with different constant shear and PSF
nfiles    =  10
#nfiles   = 200

### Galsim file names and directories

In [6]:
# Config files
galsim_config_fname     = 'csc_multishear.yaml'
galsim_config_psf_fname = 'csc_psf.yaml'
galsim_config_dir       = 'config/galsim'

# In- and outout directories
galsim_input_dir       = 'data/galsim/great3/control/space/constant'
galsim_output_base_dir = 'output/galsim'

# Image file name formats
galsim_output_gal_fname_format = 'image-%03d-%1d.fits'
galsim_output_psf_fname_format = 'starfield_image-%03d-%1d.fits'

## Main program

In [7]:
# Create g_dict, dictionary of shear values with step tuples as keys.
g_dict = {}
for step in g_steps:
    g_dict[step] = (step[0] * dg, step[1] * dg)
g_values = g_dict.values()

### Create images with galsim

In [8]:
# Set paths
galsim_config_path     = '{}/{}'.format(galsim_config_dir, galsim_config_fname)
galsim_config_psf_path = '{}/{}'.format(galsim_config_dir, galsim_config_psf_fname)

# Call galsim
create_all_sims_great3(g_values, galsim_config_path, galsim_config_psf_path, \
        galsim_input_dir, galsim_output_base_dir, galsim_output_gal_fname_format, \
        galsim_output_psf_fname_format, nxy_tiles=nxy_tiles, nfiles=nfiles, job=job)

*** Start create_all_sims_great3 ***
For shear (0.0,0.02), all images already exist, skipping galsim call
For shear (0.0,-0.02), all images already exist, skipping galsim call
For shear (0.02,0.0), all images already exist, skipping galsim call
For shear (0.0,0.0), all images already exist, skipping galsim call
For shear (-0.02,0.0), all images already exist, skipping galsim call
All PSF images already exist, skipping galsim call
*** End create_all_sims_great3 ***


### Measure shapes with KSB (shapelens)

In [9]:
# Set input directory (= galsim output directory) and output directory
ksb_input_base_dir  = galsim_output_base_dir
ksb_output_base_dir = 'output/shapelens'

In [10]:
# call get_shapes
all_shapes_shapelens(g_values, ksb_input_base_dir, ksb_output_base_dir, nfiles, nxy_tiles, job=job)

*** Start all_shapes_shapelens ***
keeping existing file output/shapelens/g1_0.0_g2_0.02/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_0.02/image-000-0.fits > output/shapelens/g1_0.0_g2_0.02/result-000.txt
keeping existing file output/shapelens/g1_0.0_g2_-0.02/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_-0.02/image-000-0.fits > output/shapelens/g1_0.0_g2_-0.02/result-000.txt
keeping existing file output/shapelens/g1_0.02_g2_0.0/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.02_g2_0.0/image-000-0.fits > output/shapelens/g1_0.02_g2_0.0/result-000.txt
keeping existing file output/shapelens/g1_0.0_g2_0.0/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_0.0/image-000-0.fits > output/shapelens/g1_0.0_g2_0.0/resu

### Read shear estimate results from files

In [11]:
# Set input directories = KSB and galsim output directories
results_input_base_dir  = ksb_output_base_dir
psf_input_dir           = '{}/psf/'.format(galsim_output_base_dir)

# Read files.
results = all_read_shapelens(g_dict, results_input_base_dir, psf_input_dir, nfiles)

*** Start all_read_shapelens ***
Read 50 files with measured galaxy properties including shapes.
Read 0 files with PSF shapes.
*** End all_read_shapelens ***


### Compute shear response matrix and save to file

In [13]:
# Output directory for shear response files
R_output_dir = 'output/R'

In [14]:
# Compute shear response
R = shear_response(results, dg, output_dir=R_output_dir)

*** Start shear_response ***
Saving R to file output/R
*** End shear_response ***


### Compute and print mean and std of shear response matrix

In [15]:
# If not compute above, shear response can be read from files
#R = read_R(R_output_dir)

print('mean(R)')
print(R.mean(axis=2))

print('std(R)')
print(R.std(axis=2))

mean(R)
[[ 0.94035822 -0.00808644]
 [ 0.00176651  0.94069214]]
std(R)
[[0.24473969 0.08859729]
 [0.08682244 0.27826104]]


### Plot shear response

In [16]:
# Variable on x-axis
xvar  = results[(0, 0)].sn

# Variable name = xlabel
xname = 'SNR'

# Number of x-bins
nbins = 3

# Variables on y-axis, can be array for multiple curves in plot
yvar   = [shear_bias_m(R, i) for i in [0, 1]]
yvar.append(R[0,1])

# Variable names for legend
yname  = ['$m_1$', '$m_2$', '$R_{12}$']

# Color of points and error bars
color  = ['g', 'r', 'b']

# Point types
marker = ['o', 's', 'd']

# Create plot and save to file
x_mean, y_mean, y_std = plot_mean_per_bin_several(xvar, xname, yvar, yname, nbins, error_mode='std', \
        color=color, lw=2, marker=marker, out_name='snr_m1.pdf')

Plot saved to snr_m1.pdf
